In [ ]:
# @title ## 2.1 Batch Preprocessing of Receptor Files 🏢 (Required)
# @markdown a. Please **click the Run button** ▶️ on the left side of the code cell and follow any prompts.
# @markdown
# @markdown b. After **uploading your files** 📤, the script will automatically process them 🔄.
# @markdown
# @markdown c. Once processing is completed ✅, the script will package the processed files 📦 and provide a download link 📥.
from google.colab import files
import os
import sys
import subprocess

!pip2 install numpy --upgrade

exec(open('/content/env_settings.py').read())

def process_receptor_files():
    print("📤 Please upload receptor files (.pdb format)...")
    uploaded = files.upload()

    if not uploaded:
        print("❌ Error: No files uploaded. Please re-upload the receptor files.")
        return

    output_directory = 'temp_receptors/'
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)

    for filename in uploaded.keys():
        receptor_path = filename
        print(f"✅ Upload successful: {receptor_path}")

        # 1. Read  receptor file
        print(f"🔄 Processing {receptor_path}...")
        result = subprocess.run([mgltools_path, prepare_receptor4_path, "-r", receptor_path, "-o", output_directory + receptor_path + "qt", "-A", "hydrogens", "-U", "nphs_lps_waters"], capture_output=True, text=True, env=os.environ)
        if result.returncode != 0:
            print(f"❌ Error: An error occurred while processing the file {receptor_path}. Error message: {result.stderr}")
            continue
        print(f"✅ Initial processing of {receptor_path} completed.")

        # 2. Add hydrogen atoms
        result = subprocess.run([mgltools_path, prepare_receptor4_path, "-r", output_directory + receptor_path + "qt", "-A", "hydrogens"], capture_output=True, text=True, env=os.environ)
        if result.returncode != 0:
            print(f"❌ Error: An error occurred while adding hydrogen atoms to {output_directory + receptor_path + 'qt'}. Error message: {result.stderr}")
            continue
        print(f"✅ Hydrogen atoms added to {receptor_path}.")

        # 3. Add charges
        result = subprocess.run([mgltools_path, prepare_receptor4_path, "-r", output_directory + receptor_path + "qt", "-A", "charges"], capture_output=True, text=True, env=os.environ)
        if result.returncode != 0:
            print(f"❌ Error: An error occurred while adding charges to {output_directory + receptor_path + 'qt'}. Error message: {result.stderr}")
            continue
        print(f"✅ Charges added to {receptor_path}.")

    # Package processed files
    print("📦 Packaging processed files...")
    !zip -r processed_receptors.zip $output_directory

    # Provide download link
    print("📥 Providing download link...")
    files.download('processed_receptors.zip')

    print("🎉 All receptor files processed successfully.")

process_receptor_files()